# Rice Dataset

In questo notebook spenderemo alcune parole su un dataset e su come interpretarlo


Questo notebook nasce da un progetto per un esame universitario che mi ha lasciato molto amaro in bocca.
Parleremo di un dataset che ho dovuto analizzare e di alcune incongruenze che sono state chiarite molto tardi


Per ora partiamo importando le librerie utili e impostando i comandi che ci interessano

In [ ]:
import matplotlib.pyplot as plt 
from matplotlib.animation import FuncAnimation
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams.update({"text.usetex": True})

import numpy as np
import pandas as pd

from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score

import warnings
warnings.filterwarnings('ignore')

import copy


## Descrizione del dataset
Charichiamo il dataset.
Il dataset è un dataset molto famoso, presenta 3810 osservazioni di chicchi di riso appartenenti a due famiglie differenti: _Osmancik_ e _Cameo_. Tutti i dati sono stati raccolti in maniera automatica da un processo di analisi di una fotocamera dei chicci di riso. Infatti tutte le variabili quantitative sono misurate in pixels. In particolare le variabili presenti nel dataset sono
- **Area**               
- **Perimeter**
- **Major_Axis_Length**
- **Minor_Axis_Length**
- **Eccentricity**:  Misura della sfericità del chicco. Assume valori tra $0$ ed $1$ dove zero indica un cerchio perfetto ed uno due rette che non si incontrano mai. (ci torneremo in seguito)
- **Convex_Area**: Più piccola regione convessa che comprende il chicco. Di fatto un approssimazione per eccesso dell'area saltando le scalanature e le rientranze del chicco     
- **Extent**: rapporto tra l'area del chicco ed il riquadro che lo contiene
- **Class**: famiglia di appartenenza del chicco

Il fatto che le variabili siano ricavate in pixel meriterebbe un discorso più approfondito sul tema della discretizzazione delle variabili. Questo non è lo spazio dove dedicarglielo ed inoltre come vedremo a breve i valori assoluti delle variabili sono molto elevati quindi possiamo assumere una buona approssimazione delle variabili anche se sono discretizzate.

In [ ]:
link = 'https://archive.ics.uci.edu/static/public/545/data.csv'

df = pd.read_csv(link)
df['Class'].replace("Cammeo","Cammeo ") #Per semplici motivi estetici rendo le stringhe 

unit = {
    "Area":"px",
    "Perimeter":"px",
    "Major_Axis_Length":"",
    "Minor_Axis_Length":"",
    "Eccentricity":"",
    "Convex_Area":"px",
    "Extent":"",
}

In [ ]:
df.sample(5)

Le informazioni presente dalla fonte sul dataset sono poche ed ambigue. Su come sia stata misurata l'Eccentricity, sull'Extent in generale.

In [ ]:
df.info()

In [ ]:
df.describe()

Cerchiamo anche di visualizzare queste variabili con un simpatico grafichino

In [ ]:
def eggshape(a=1.5,b=1,c=0.2, n_points = 200):
    # Probabilemte ho sbagliato qualcosina nell'equazione del uovo rispetto al parametro "b"
    # Però mi sembra l'ultimo dei problemi
    plt.figure(figsize=(8,8))
    pos = lambda x: b*((1-x**2/a**2)*b**2/(1+c*x))**0.5
    neg = lambda x: -b*((1-x**2/a**2)*b**2/(1+c*x))**0.5
    X = np.linspace(-a,a,n_points)
    Y_a = [pos(x) for x in X]
    Y_b = [neg(x) for x in X]
    asse_x = (-2+(4-4*a**2*c**2)**0.5) /(2*c)
    
    # Perimetro
    plt.plot(X,Y_a, color = "black", label = "Perimeter")
    plt.plot(X,Y_b, color = "black")
    
    #minor 
    plt.plot([asse_x,asse_x],[neg(asse_x),pos(asse_x)],color = 'red', linestyle = '-', 
             label ='Minor_Axis_Length', alpha = 0.5)

    # major
    plt.plot([-a,a],[0,0],color = 'darkorange', linestyle = '-', 
             label ='Major_Axis_Length', alpha = 0.5)
    # Area
    plt.fill_between(x= X, y1= Y_a, y2 = Y_b, color= "g", alpha= 0.15, label ='Area')
    
    # Eccentricity
    r = pos(asse_x)
    x_c = np.linspace(asse_x-r,asse_x+r,200)
    plt.plot(x_c, [(r**2-(x-asse_x)**2)**0.5 for x in x_c] , linestyle='--', color = 'green', 
             label = 'Eccentricity')
    plt.plot(x_c, [-(r**2-((x-asse_x))**2)**0.5 for x in x_c], linestyle='--', color = 'green')
    
    plt.title("Variable Visualization")
    
    plt.xlim([min(neg(asse_x),-a) * 1.1, max(pos(asse_x),a) * 1.1])
    plt.ylim([min(neg(asse_x),-a) * 1.1, max(pos(asse_x),a) * 1.1])
    plt.legend(loc='upper right')
    plt.grid(True)
    plt.tick_params(axis='both', which='both', bottom=False, top=False, left=False, right=False)
    # plt.gca().set_xticklabels([])
    # plt.gca().set_yticklabels([])

    plt.show()

eggshape( c= 0.00001)


## Analisi Statistiche
Procediamo guardando come si distribuiscono e come interagiscono tra di loro queste variabili

Per farlo mi preparo qualche bella funzione per rappresentarle (che altro non era che il corpo del mio elaborato per l'uni)

In [ ]:
def interval_Gaussian_estimation(data, number_point = 50, interval:tuple = None):
    mu = np.mean(data)  
    sigma = np.var(data) ** 0.5  
    if not interval:
        interval = (min(data),max(data))
    x = np.linspace(*interval,number_point)
    y = ((1 / (np.sqrt(2 * np.pi) * sigma)) * np.exp(-0.5 * (1 / sigma * (x - mu))**2)) 
    return x,y

def bivariate_regression(X,Y):
    b = np.cov(X,Y)[0][1] / np.var(X)
    a = np.mean(Y) - b*np.mean(X)
    return a,b

def plot_corr(corr):
    if corr<0:
        col = 'red'
    else:
        col = 'black'
    if abs(corr) > 0.9:
        wgt = 'bold'
        sz = 'large'
    elif abs(corr) > 0.75:
        wgt = 'semibold'
        sz = 'large'
    elif abs(corr) > 0.25:
        wgt = 'normal'
        sz = 'medium'
    elif abs(corr) > 0.1:
        wgt = 'light'
        sz = 'small'
    else:
        wgt = 'ultralight'
        sz = 'small'
    return col, wgt, sz

# Distribution supervised and unsupervised
def global_distribution(df,classVariable = None):
    df_numeric = df.select_dtypes(include='number')
    fig, axes = plt.subplots(2,len(df_numeric.columns),figsize=(3*len(df_numeric.columns), 4))
    if classVariable:
        cmap = plt.cm.get_cmap("tab20")
        unique = df[classVariable].unique()
        colors = [cmap(i) for i in range(2*len(unique))]

        for i,c in enumerate(df_numeric.columns):
            for index,v in enumerate(unique):
                axes[0][i].hist(df[c][df[classVariable]==v],color = colors[2*index],edgecolor = colors[2*index + 1],alpha = 0.5)
            axes[0][i].set_xticks([])
            axes[0][i].set_title(c)
            if i ==0: #mostra label solo per la prima colonna
                box_plot = axes[1][i].boxplot([df[c][df[classVariable]==v] for v in unique], meanline = False, vert= False, patch_artist = True, labels = unique)
            else:
                box_plot = axes[1][i].boxplot([df[c][df[classVariable]==v] for v in unique], meanline = False, vert= False, patch_artist = True, labels = ['' for i in unique] )
            col = [colors[2*i+1] for i in range(len(unique))]
            for patch, color in zip(box_plot['boxes'], col):
                patch.set_facecolor(color)
    else:
        
        for i,c in enumerate(df_numeric.columns):
            axes[0][i].hist(df[c],color = 'grey',edgecolor = 'black')
            axes[0][i].set_xticks([])
            axes[0][i].set_title(c)
            

            axes[1][i].boxplot(df[c], vert = False)
            axes[1][i].set_yticks([])

def pairs_plot(df,classVariable = None):
    df_numeric = df.select_dtypes(include='number')
    if classVariable:
        corr = {}
        for c in df[classVariable].unique():
            corr[c] = df_numeric[df[classVariable] == c].corr()
    else:
        corr = df_numeric.corr()
    
    fig, axes = plt.subplots(len(df_numeric.columns), len(df_numeric.columns), figsize=(2*len(df_numeric.columns),2*0.618*len(df_numeric.columns))) 
    plt.subplots_adjust(hspace=1)
    if classVariable:
        cmap = plt.cm.get_cmap("tab20") # tab 20 ha i colori accopiati, ne uso uno per i punti l'altro per la retta
        colors = [cmap(i) for i in range(2*len(df[classVariable].unique()))]
        

    for i,c1 in enumerate(df_numeric.columns):
        for j,c2 in enumerate(df_numeric.columns):
            if i > j:
                if  classVariable:
                    for index,mod in enumerate(df[classVariable].unique()):
                        axes[i][j].scatter(df[c1][df[classVariable]==mod],df[c2][df[classVariable]==mod], color=colors[index*2+1],s = 0.2)
                        df1 = df[c1][df[classVariable]==mod]
                        df2 = df[c2][df[classVariable]==mod]
                        a,b = bivariate_regression(df1,df2)
                        axes[i][j].plot((min(df[c1]),max(df[c1])),(min(df[c1])*b+a,max(df[c1])*b+a),color = colors[index*2], linewidth=0.5, label = mod) # l'intervallo min_x max_x va calcolato su TUTTO il df (pensa e capisci da solo perchè pirla)
                 
                        
                    if i == 1 and j==0:
                        handles, labels = axes[i][j].get_legend_handles_labels()
     
                else:
                    axes[i][j].scatter(df_numeric[c1],df_numeric[c2], s = 0.2, color = 'black')
                    # regretion
                    b = (np.mean([a*b for a,b in zip(df[c1],df[c2])]) - np.mean(df[c1]) * np.mean(df[c2])) / (np.mean([x**2 for x in df[c1]]) - np.mean(df[c1])**2)
                    a = np.mean(df[c2]) - b*np.mean(df[c1])
                    axes[i][j].plot((min(df[c1]),max(df[c1])),(min(df[c1])*b+a,max(df[c1])*b+a),color = 'red', linewidth=0.5)
                axes[i][j].set_xticks([])
                axes[i][j].set_yticks([])
                
            elif i == j:
                axes[i][j].set_title(c2,fontdict = {'fontsize':10})
                if classVariable:
                    interval = (min(df_numeric[c1]),max(df_numeric[c1]))
                    for index,c in enumerate(df[classVariable].unique()):
                        # Istogramma
                        axes[i][j].hist(df_numeric[df[classVariable] == c][c1],density = True, color = colors[index*2+1], edgecolor = colors[index*2], alpha = 0.7)
                        
                        # Gaussiana
                        axes[i][j].plot(*interval_Gaussian_estimation(df_numeric[df[classVariable] == c][c1],interval = interval), '--',color = colors[index*2], linewidth=1)
                        axes[i][j].set_yticks([])
      
                    
                else:
                    axes[i][j].hist(df_numeric[c1],color = 'grey',density = True, edgecolor = 'black')
                    
                    axes[i][j].plot(*interval_Gaussian_estimation(df_numeric[c1]), '--',color = 'red', linewidth=1)

                    axes[i][j].set_yticks([])
      
            elif i < j:
                axes[i][j].set_xlim((0,1))
                axes[i][j].set_ylim((0,1))
  
                axes[i][j].axis('off')
                
                if classVariable:
                    n_mod = len(df[classVariable].unique())*2 
                    for index,c in enumerate(df[classVariable].unique()):
                        colore, peso, dimensione = plot_corr(corr[c][c1][c2])
                        axes[i][j].scatter([0.2],[(index*2+1)/n_mod], color = colors[index*2])
                        axes[i][j].text(0.3,(index*2+1)/n_mod,str(round(corr[c][c1][c2],3)),size = dimensione, weight = peso, color = colore, horizontalalignment='left', verticalalignment='center')
                
                else:
                    colore, peso, dimensione = plot_corr(corr[c1][c2])
                    axes[i][j].text(0.5,0.5,str(round(corr[c1][c2],3)), size = dimensione, weight = peso, color = colore, horizontalalignment='center', verticalalignment='center')
                         

    if classVariable:
        fig.suptitle(f'Correlation Plot {classVariable}',weight = 'semibold')
        fig.legend(handles, labels, loc = 'lower center')
        
    else:
        fig.suptitle('Correlation Plot',weight = 'semibold')
    #plt.savefig(f'image/Correplation Plot {classVariable}.png')
    plt.show()
    
def distribution(df,explainVariable ,classVariable = None):
    """
    Questa funzione prende tre parametri
    - df:   
    """
    fig, axes = plt.subplots(2,1,figsize=(8, 6),sharex=True)

    if classVariable:
        cmap = plt.cm.get_cmap("tab20")
        unique = df[classVariable].unique()
        colors = [cmap(i) for i in range(2*len(unique))]

        for index,v in enumerate(unique):
            axes[0].hist(df[explainVariable][df[classVariable]==v],color = colors[2*index],edgecolor = colors[2*index + 1],alpha = 0.5)

        axes[0].set_title(explainVariable)
        box_plot = axes[1].boxplot([df[explainVariable][df[classVariable]==v] for v in unique], meanline = False, vert= False, patch_artist = True, labels = unique)
        col = [colors[2*i+1] for i in range(len(unique))]
        for patch, color in zip(box_plot['boxes'], col):
            patch.set_facecolor(color)
    else:
        axes[0].hist(df[explainVariable],color = 'grey',edgecolor = 'black')
        axes[0].set_title(explainVariable)
        
        axes[1].boxplot(df[explainVariable], vert = False)
        axes[1].set_yticks([])
        
def bivariate_regression(X,Y):
    b = np.cov(X,Y)[0][1] / np.var(X)
    a = np.mean(Y) - b*np.mean(X)
    return a,b

def bivariate(df,variable_a,variable_b, class_variable = None):
    plt.figure(figsize=(12,6))
    if class_variable:
        cmap = plt.cm.get_cmap("tab20")
        unique = df[class_variable].unique()
        colors = [cmap(i) for i in range(2*len(unique))]

        for index,mod in enumerate(df[class_variable].unique()):
            df1 = df[variable_a][df[class_variable]==mod]
            df2 = df[variable_b][df[class_variable]==mod]
            
            #Scatter
            plt.scatter(df1,df2, color=colors[index*2+1],s = 0.5)
            
            
            #Regression
            a,b = bivariate_regression(df1,df2)
            R_2 = round(np.corrcoef(df1,df2)[0][1]**2,2)
            if b > 0 :
                adding_s = '+'
            else:
                adding_s ='-'
            equation = f"{mod}: $y = {round(a,2)} {adding_s} {round(abs(b),2)} x$\n{'  '*len(mod)}  $R^2={R_2}$"
            plt.plot((min(df[variable_a]),max(df[variable_a])),(min(df[variable_a])*b+a,max(df[variable_a])*b+a),color = colors[index*2], linewidth=0.5, label = equation) # l'intervallo min_x max_x va calcolato su TUTTO il df (pensa e capisci da solo perchè pirla)
            
            # Center point
            centerPoint = (np.mean(df1), np.mean(df2))
            plt.scatter(*centerPoint,marker = 'x',color = colors[index*2])
       
    

    else:
        #Scatter
        plt.scatter(df[variable_a],df[variable_b], s = 1, color = 'dimgray')
        
        #Regression
        a,b = bivariate_regression(df[variable_a],df[variable_b])
        R_2 = round(np.corrcoef(df[variable_a],df[variable_b])[0][1]**2,2)
        if b > 0 :
            adding_s = '+'
        else:
            adding_s ='-'
        #equation = f"{variable_b} = {round(a,2)}  {round(b,2)} {variable_a}   {R_2}"
        equation = f"$y = {round(a,2)} {adding_s} {round(abs(b),2)} x$\n$R^2={R_2}$"
        plt.plot((min(df[variable_a]),max(df[variable_a])),(min(df[variable_a])*b+a,max(df[variable_a])*b+a),color = 'red', linewidth=0.5, label = equation)
        
        # Center point
        centerPoint = (np.mean(df[variable_a]), np.mean(df[variable_b]))
        plt.scatter(*centerPoint,marker="x",color = "black")
        
        # xticks = plt.get_xticks()
        # xticks.append(centerPoint[0])
        
        
    
    plt.xlabel(f"{variable_a}")
    if variable_a in unit:
        plt.annotate(f"{unit[variable_a]}", xy=(1, -0.05), xycoords="axes fraction")
    if variable_b in unit:
        plt.annotate(f"{unit[variable_b]}", xy=(-0.05, 1), xycoords="axes fraction")
    plt.ylabel(f"{variable_b}")
    plt.legend()
    plt.title(f'{variable_b}  -  {variable_a}')
    #plt.savefig(f"image/{variable_b}-{variable_a}-{class_variable}.png")
    plt.show()
    

Vediamo un po'

In [ ]:
global_distribution(df[df.columns[:len(df.columns)//2]])
global_distribution(df[df.columns[len(df.columns)//2:]])

Notiamo che quasi tutte le variabili sembrano avere distribuzione bimodale e forme non particolarmente chiare, ma se provaimo a differenziare sulla base del tipo di riso cosa otteniamo?

In [ ]:
global_distribution(df[['Class'] + list(df.columns[:len(df.columns)//2])],"Class")
global_distribution(df[df.columns[len(df.columns)//2:]],"Class")

Ecco che la bimodalità e le forme strane vengono totalmente spiegate. La bimodalità è dovuta al fatto che osserviamo due distribuzioni differenti sovrapposte, se impariamo a differenziarle ecco che tutto diventa chiaro. Tranne per _Extent_ che mantiene una bimodalità e non sembra essere discrimanto per nessuna maniera rispetto alla famiglia di riso. Guardiamolo un po' più nel dettaglio

In [ ]:
distribution(df,"Extent","Class")

Nessuna differenza sostanziale tra le due famiglie, due mode intorno a _0.63_ e _0.78_

Procediamo studiando il caso bivariato e vediamo se emerge qualcosa di interessante.

In [ ]:
pairs_plot(df)
pairs_plot(df,'Class')

Notiamo tante correlazionoi positive, che non cambaino tra i livelli delle famiglie. I chicchi di riso sembrano differire nei valori delle variabili ma non in come esse si relazionano tra di loro. Tecniche semplici come un'anova potrebbero già portarci a buone inferenze sulla famiglia di appartenenza di un chicco. Ma _Extent_ continua ad avere un comportamento ambiguo. Totalmente incorrelata con il resto. Ma qui non è più normale e se vogliamo essere dei bravi statistici non possiamo chiudere gli occhi. Quello che questi grafici ci dicono è che ci sono chicchi di riso che hanno tutte le variabili simili (poichè correlate) ed _Extent_ diverso. Come è possibile?
Ricordiamo che _Extent_ è il rapporto tra l'area del chicco e l'area del rettangolo che lo contiene. Concettualmente non è possibile che chicchi simili in tutto abbiano _Extent_ diverso. Nessuna idea?

In [ ]:
print(df.loc[1971],df.loc[3584],sep = '\n\n')

Un esempio di due righe con dati molto simili ed extent diverso.
il primo chicco dovrebbe essere contenuto in un rettangolo di $\frac{8499}{0.761559} = 11160$
mentre il secndond dovrebbe stare in un rettangolo di $\frac{8501}{0.639846} = 13286$

Una differenza enorme se si considera che le aree differiscono di due pixel su 8500, i perimetri di un pixel su 370 gli assi di 3 pixel su 150 ed 1 su 70.

Proviamo a ricavare in generale l'ipotetica area del rettangolo e vediamo come si comporta

In [ ]:

df['BoxArea'] = df['Area'] / df['Extent']

pairs_plot(df[["BoxArea","Area","Extent","Class"]],"Class")
bivariate(df,"Area","BoxArea","Class")

del df['BoxArea'] #Togliamola perchè dopo non ci servirà e ci confonde solo

Direi un esito inaspettato.

Per risolvere questo arcano dobbiamo ricordare che il dataset è costruito in maniera automatica dall'analisi fotografica dei chicci. Quindi per rettangolo intendono il rettangolo di pixel, senza tener conto dell'orientamento del chicco di riso nell'immagine. In poche parole abbiamo una variabile che dipende totalmente dall'angolo di rotazione del chicco nell'immagine.


Ora entrariamo nella parte più matematica della faccenda: assumeremo il chicco di riso come un'ellisse perfetto (discuteremo poi della bontà di questa approssimazione) e vedremo se questa intepretazione riesce a spiegare il comportamento del nostro extent


# Bounding Box di un ellisse ruotato

La formula matematica che definisce un ellise è l'insieme dei punti che rispettano l'equazione 
$$\left(\frac{x}{a}\right)^2 +\left(\frac{y}{b}\right)^2 = 1$$

Dove $a$ determina quanto l'ellisse è allungato in orizzontale, e coincide con la metà dell'asse orizzontale. Mentre $b$ detdetermina quanto l'ellisse è allungato in verticale, e coincide con la metà dell'asse verticale.

Definiamo quindi 
$$Ellisse(a,b) := \set{(x,y) : \left(\frac{x}{a}\right)^2 +\left(\frac{y}{b}\right)^2 = 1}\tag{0}$$


Questo per lo meno è un ellisse con gli assi adagiati sulle rette $x = 0$ e $y = 0$. Ma poichè noi vogliamo proprio l'esito della rotazione di questo ellisse dobbiamo introdurre le rotazioni.

Ricordando che la matrice di rotazione rispetto all'angolo $\theta$
$$
\begin{bmatrix} 
\cos(\theta) &-\sin(\theta) \\
\sin(\theta) & \cos(\theta)
\end{bmatrix}

\begin{bmatrix} 
x \\
y
\end{bmatrix}
=
\begin{bmatrix} 
x \cos(\theta) -y\sin(\theta) \\
x \sin(\theta) +y\cos(\theta)
\end{bmatrix}
$$

Possiamo introdurre un ellisse ruotato di $\theta$ radianti come

$$Ellisse\_R(a,b,\theta) := \set{(x',y') : x' = x \cos(\theta) -y\sin(\theta)y;\space y'=x \sin(\theta) +y\cos(\theta)\space\forall(x,y)\in Ellisse(a,b)}\tag{1}$$

Adesso che stiamo ragionando con tutte le possibili rotazioni possiamo assumere senza perdità di generalità $a \ge b$

A noi di questo ellisse ruotato interessa trovare l'area del più piccolo rettangolo che lo contiene. In altre parole dobbiamo trovare, in funzione di $\theta$: $x_{max},x_{min},y_{max},y_{min}$

Ok visto che gli statistici solitamente non sono dei gran fan della matematica pura, scriviamo qualche bella funzione per visualizzare quanto detto fin'ora e partiamo con un approccio empirico. 

In [ ]:
def get_ellipse(a, b, n=1000):
    assert a >= b
    X = np.linspace(-a, a, n)
    Y_a = [b * (1 - (x / a) ** 2) ** 0.5 for x in X]
    Y_b = [-y for y in Y_a]
    return list(zip(X, Y_a)), list(zip(X, Y_b))

def rotate(X, Y, theta):
    X_1 = [np.cos(theta) * x - np.sin(theta) * y for x, y in zip(X, Y)]
    Y_1 = [np.sin(theta) * x + np.cos(theta) * y for x, y in zip(X, Y)]
    return X_1, Y_1

def getArea(X, Y, X_, Y_, t):
    X_1, Y_1 = rotate(X, Y, t)  
    X_2, Y_2 = rotate(X_, Y_, t)
    return (max(X_1 + X_2) - min(X_1 + X_2)) * (max(Y_1 + Y_2) - min(Y_1 + Y_2))


a,b = 3,1
A, B = get_ellipse(3,1)

X, Y = list(zip(*A))

X_, Y_ = list(zip(*B))
fig,axes = plt.subplots(1,2,figsize = (10,5))
axes[0].plot(X ,Y , color = 'blue')
axes[0].plot(X_,Y_, color = 'blue')
axes[0].plot([min(X + X_),max(X + X_)],[min(Y + Y_),min(Y + Y_)], color = "blue")
axes[0].plot([min(X + X_),max(X + X_)],[max(Y + Y_),max(Y + Y_)], color = "blue")
axes[0].plot([min(X + X_),min(X + X_)],[min(Y + Y_),max(Y + Y_)], color = "blue")
axes[0].plot([max(X + X_),max(X + X_)],[min(Y + Y_),max(Y + Y_)], color = "blue")

axes[0].set_xlim(-a,a)
axes[0].set_ylim(-a,a)
axes[0].axis('off')

X_1, Y_1 = rotate(X, Y, 0.5)
X_2, Y_2 = rotate(X_, Y_, 0.5)
axes[1].plot(X_1 ,Y_1, color = 'red')
axes[1].plot(X_2,Y_2, color = 'red')
axes[1].plot([min(X_1 + X_2),max(X_1 + X_2)],[min(Y_1 + Y_2),min(Y_1 + Y_2)], color = "red")
axes[1].plot([min(X_1 + X_2),max(X_1 + X_2)],[max(Y_1 + Y_2),max(Y_1 + Y_2)], color = "red")
axes[1].plot([min(X_1 + X_2),min(X_1 + X_2)],[min(Y_1 + Y_2),max(Y_1 + Y_2)], color = "red")
axes[1].plot([max(X_1 + X_2),max(X_1 + X_2)],[min(Y_1 + Y_2),max(Y_1 + Y_2)], color = "red")
axes[1].set_xlim(-a,a)
axes[1].set_ylim(-a,a)
axes[1].axis('off')
plt.show()

Facciamo anche una bella animazione per non farci mancare niente

In [ ]:

a,b = 2.25 ,1
A, B = get_ellipse(a,b)
X, Y = list(zip(*A))
X_, Y_ = list(zip(*B))

fig = plt.figure(figsize=(15, 6))
ax = fig.add_subplot(1,2,1)
line1, = ax.plot([], [], color="blue")
line2, = ax.plot([], [], color="blue")
line3, = ax.plot([], [], color="red")
line4, = ax.plot([], [], color="red")
line5, = ax.plot([], [], color="red")
line6, = ax.plot([], [], color="red")


area = fig.add_subplot(1,2,2)
temp, = area.plot([],[], color ='red')

theta_data = []
area_data = []


def init():
    line1.set_data([], [])
    line2.set_data([], [])
    line3.set_data([], [])
    line4.set_data([], [])
    line5.set_data([], [])
    line6.set_data([], [])
    temp.set_data([],[])
    return line1, line2, line3, line4, line5, line6, temp

def update(frame):
    global theta_data, area_data
    theta = frame * np.pi / 180  # Converti il frame in radianti
    X_1, Y_1 = rotate(X, Y, theta)
    X_2, Y_2 = rotate(X_, Y_, theta)
    line1.set_data(X_1, Y_1)
    line2.set_data(X_2, Y_2)
    line3.set_data([min(X_1 + X_2),max(X_1 + X_2)],[min(Y_1 + Y_2),min(Y_1 + Y_2)])
    line4.set_data([min(X_1 + X_2),max(X_1 + X_2)],[max(Y_1 + Y_2),max(Y_1 + Y_2)])
    line5.set_data([min(X_1 + X_2),min(X_1 + X_2)],[min(Y_1 + Y_2),max(Y_1 + Y_2)])
    line6.set_data([max(X_1 + X_2),max(X_1 + X_2)],[min(Y_1 + Y_2),max(Y_1 + Y_2)])
    theta_data.append(theta)
    area_data.append((max(X_1 + X_2) - min(X_1 + X_2)) * (max(Y_1 + Y_2) - min(Y_1 + Y_2)))
    temp.set_data(theta_data,area_data)
    return line1, line2, line3, line4, line5, line6, temp

ax.set_xlim(-a - 1, a + 1)
ax.set_ylim(-a - 1, a + 1)

area.set_xlim(0, np.pi*2)
area.set_ylim(4 * a * b -1, 2*(a**2+b**2) + 1) # estremi calcolati analiticamente
#area.set_ylim(m-1, 2 * a**2 + 3) prima utilizzavo questi estremi poichè l'area è massmio a 45° la diagonale del Q è circa 2a, dunque 2 * a**2 ≈ area max!

ax.set_title('Rect and Box')
area.set_title('Area over angle')
area.set_xticks([0,np.pi * 0.25,np.pi * 0.5, np.pi * 0.75, np.pi * 1,np.pi * 1.25,np.pi * 1.5,np.pi * 1.75, np.pi * 2])
area.set_xticklabels( ['0', '$\pi$/4','$\pi$/2','3/4 $\pi$','$\pi$','5/4 $\pi$','3/2 $\pi$','7/4 $\pi$','2$\pi$'])

ani = FuncAnimation(fig, update, frames=np.arange(0, 360, 1), init_func=init, blit=True)
# ani


Questo andamento spiega la bimodalità dell'extent e perchè è indipendente dal tipo di chicco, infatti invertendo la funzione otteniamo l'istogramma bimodale da cui, con un po' di rumore, deriva la nostra osservazione

In [ ]:
thetas = np.linspace(0,np.pi * 0.25,1000) # basta l'intervallo 0 - π/4 per ciclicità. Poi posso invertire la funzione
Area = a * b * np.pi
extents = [ Area / getArea(X,Y,X_,Y_,t) for t in thetas]


fig, axis = plt.subplots(1,2,figsize = (15,5))
axis[0].hist(extents, bins = 30, density = True, edgecolor = 'black', alpha = 0.7) 
axis[0].set_xlabel('Extent')
axis[0].set_ylabel('Frequency')
axis[0].set_yticks([])
axis[0].set_title('Teorico')



axis[1].hist(df["Extent"], bins = 30, density = True, edgecolor = 'black', alpha = 0.7) 
axis[1].set_xlabel('Extent')
axis[1].set_yticks([])
axis[1].set_title('Dataset')


plt.show()


L'extent massimo è proprio $\frac{\pi}{4}$ e non dipende dalla forma dell'ellisse. Questo spiega il picco a $0.785\approx \frac{\pi}{4}$.
Mentre l'extent minimo dipende dal tipo di ellisse, ma vediamo che in generale è più frequente di quello massimo. Questo spiega il picco maggiore ed anche perchè siano indipendenti dalla famiglia di riso.


Questo è forse un buon momento per introdurre un aspetto legato ai dati molto interessante. Si è notato prima come tutti i dati sono fortemente correlati positivamente, (fatta eccezzione di Extent). Questo non è troppo positivo, vuol dire che l'informazione è molto ridondante. Ogni variabile Ha press'appoco lo stesso significato delle altre. Più o meno tutte si rifanno a _forma_ o _dimensione_ del chicco. 

Questo concetto possiamo visualizzarlo molto bene in molte maniere, io ne userò due:
1. Uno scatter plot tridimensionale delle variabili ci mostra come esse stiano di fatto su un piano, dunque conoscendone due si può ricavare la terza, non c'è alcun contributo informativo nella terza che non si potesse indurre dalle prime due. (Negativo quando la terza non è quella che vogliamo siegare)
2. Vedendo quanto spiegano le componenti principali, e soprattuto vedere quanto cambia la presenza o assenza dell'extent

Inoltre già che calcoliamo le Componenti Principali approfittiamone per confrontarle con le Componenti Discriminanti, esce fuori una riflessione molto carina.

In [ ]:
def update(i):
  # Calcolo angolo di rotazione
  n = i * 0.3  # Modifica fattore per velocità di rotazione

  # Aggiornamento vista 3D
  ax.view_init(azim=n, elev=0.8*n)

  # Ritorno del grafico scatter (necessario per FuncAnimation)
  return scat

# Figura e assi
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

# Creazione grafico scatter

for c in df['Class'].unique():
  scat = ax.scatter(df['Major_Axis_Length'][df['Class'] == c], df['Minor_Axis_Length'][df['Class'] == c], df['Area'][df['Class'] == c], s=0.5)

# Impostazione etichette assi
ax.set_xlabel('Major_Axis_Length')
ax.set_ylabel('Minor_Axis_Length')
ax.set_zlabel('Area')
ax.view_init(azim=5, elev=4)
# Animazione
anim = FuncAnimation(fig, update, frames=300, interval=10) 
# anim

In [ ]:
# Visualizzare la separazione lineare

def get_pca(df, n = 3):
    df_numeric = df.select_dtypes(include='number')
    df_st = (df_numeric - df_numeric.mean(axis=0)) / df_numeric.std(axis=0)
    pca = PCA(n_components=len(df_st.columns))
    pca.fit(df_st)
    pc_scores = pca.transform(df_st)
    for i in range(1,4):
            print(f'Varianza spiegata dalle prime {i} PC:',round(pca.explained_variance_ratio_[:i].sum(),4))

def show_linear_separation(df):
    plt.figure(figsize=(10,6))
    df_numeric = df.select_dtypes(include='number')
    df_numeric = (df_numeric - df_numeric.mean(axis=0)) / df_numeric.std(axis=0)


    pca = PCA(n_components=1)
    pca.fit(df_numeric)
    pc_scores = pca.transform(df_numeric)



    lda = LinearDiscriminantAnalysis(n_components=1)
    lda.fit(df_numeric, df['Class'])
    lda_scores = lda.transform(df_numeric)

    df_numeric['LD1'] = lda_scores[:, 0]
    df_numeric['PC1'] = pc_scores[:, 0]


    for c in df['Class'].unique():
        plt.scatter(df_numeric['LD1'][df['Class'] == c],df_numeric['PC1'][df['Class'] == c], label = c ,s = 0.5)   

    r = round((df_numeric[[ 'LD1' , 'PC1' ]].corr()["LD1"]["PC1"])**2,3)

    plt.scatter([0,0],[0,0], alpha = 0, label = f'$\\rho^2 = {r}$')
    plt.legend()
    plt.title('Linear Separation')
    plt.xlabel('LD 1')
    plt.ylabel('PC 1')
    plt.legend()
    plt.show()



show_linear_separation(df)

print('Totale:')
get_pca(df)
print("\nSenza Extent:")
get_pca(df.drop("Extent",axis = 1))


c.v.d.  
Questo vuol dire che Extent da solo porta un sacco di informazione indipendente dalle altre variabili. Ed è vero, lo intuivamo già dal pair plot. Ma questa informazione è letteralmente l'informazione su "con che angolo sono orientati i chicchi nell'immagine". Cioè un'informazione di cui non ci frega una sega. Da un lato è vero che aggiungere una variabile non può fare danno, di fatto stiamo solo aumentando lo spazio dove cercare soluzioni, quindi sicuramente troveremo una soluzione migliore (nel training) ma rischiamo maggiormente l'overfitting. Questo lo valuteremo e lo discuteremo meglio in seguito

Non approfondisco qui il tema dell'informazione contenuta in una variabile espressa come variazione della varianza spiegata cumulata delle componenti principali perchè sarà il tema di un prossimo articolo.

Torniamo ora però alla parte matematica e cerchiamo di ricavare analiticamente la funzione che esprime l'Area della bounding box rispetto l'angolo di rotazione.


Dobbiamo trovare, in funzione di $\theta$: $x_{max},x_{min},y_{max},y_{min}$. Si può facilmente dimostrare che $x_{min} = - x_{max}$ e $y_{min} = -y_{max}$ ma ci rimangono comunque da calcolare i due massimi. Per farlo dobbiamo ricondurci ad una forma analitica dell'ellise.


Dalla $(0)$ otteniamo che
$$y=\pm b\sqrt{1-\left( \frac{x}{a} \right)^2}$$
e di conseguenza dalla $(1)$
$$x' = \cos(\theta)x\pm\sin(\theta)b\sqrt{1-\left( \frac{x}{a} \right)^2}$$
$$y' = \sin(\theta)x\pm\cos(\theta)b\sqrt{1-\left( \frac{x}{a} \right)^2}$$

Dalle simmetrie dell'ellisse possiamo evitare di studiarlo per $\theta > \frac{\pi}{2}$. (Come si vede dall'animazione già da $\frac{\pi}{4}$ abbiamo simmetrie, ma questo lo dimostreremo solo in seguito.)

Dunque dato che $\sin(x)$ è poisitivo in $[0,\frac{\pi}{2}]$ abbiamo che $x_{max} = \max \left({\cos(\theta)x+\sin(\theta)b\sqrt{1-\left( \frac{x}{a} \right)^2}}\right)$

Che con un po' di travaglinate ci porta a
$$x =\frac{a^2\cos(\theta)}{\sqrt{a^2\cos^2(\theta)+b^2\sin^2(\theta)}}$$
e di conseguenza, con altrettante travaglinate
$$x'_{max} =\sqrt{a^2\cos^2(\theta)+b^2\sin^2(\theta)}$$
in maniera analoga si arriva a dimostrare 
$$y'_{max}=\sqrt{b^2\cos^2(\theta)+a^2\sin^2(\theta)}$$

Dunque possiamo finalmente esprimere l'area in funzione dell'angolo
$$
\begin{align}
A&=(x_{max}-x_{min})(y_{max}-y_{min}) \\
&=2\sqrt{a^2\cos^2(\theta)+b^2\sin^2(\theta)}2\sqrt{b^2\cos^2(\theta)+a^2\sin^2(\theta)}= \\
&=4\sqrt{a^2\cos^2(\theta)+b^2\sin^2(\theta)}\sqrt{b^2\cos^2(\theta)+a^2\sin^2(\theta)}
\end{align}
$$

Proviamo ad invertire questa formula, perchè?
Beh ma perchè ora sappiamo ricavare l'area di una bounding box da un angolo ma noi la bounding box la sappiamo già. Noi vogliamo poter ricavare l'ipotetico angolo del chicco di riso nella nostra immagine, così da poter fare un po' di test d'inferenza. 

Ricordando le due identità trigonometriche
- $\sin(x)^2=\frac{1}{2}\left[ 1-\cos(2x) \right]$
- $\cos(x)^2=\frac{1}{2}\left[ 1+\cos(2x) \right]$ 

Possiamo ricondurre tutta la formula al solo $\cos$

$$
\begin{align} 
A&=2\sqrt{a^2\cos^2(\theta)+b^2\sin^2(\theta)}2\sqrt{b^2\cos^2(\theta)+a^2\sin^2(\theta)} \\
&=4\sqrt{a^2{\frac{1}{2}\left[ 1+\cos(2\theta) \right]}+b^2{\frac{1}{2}\left[ 1-\cos(2\theta) \right]}}\sqrt{b^2{\frac{1}{2}\left[ 1+\cos(2\theta) \right]}+a^2{\frac{1}{2}\left[ 1-\cos(2\theta) \right]}} \\
&=4\sqrt{\frac{a^2}{2}+\frac{b^2}{2}+\frac{(a^2-b^2)\cos(2\theta)}{2}}\sqrt{\frac{a^2}{2}+\frac{b^2}{2}+\frac{(b^2-a^2)\cos(2\theta)}{2}} \\
&=2\sqrt{a^2+b^2+(a^2-b^2)\cos(2\theta)}\sqrt{a^2+b^2+(b^2-a^2)\cos(2\theta)}
\end{align}
$$

Ed esprimendo l'area al quadrato togliamo tutte le radici fastidiose
$$
A^2=4\left[a^2+b^2+(a^2-b^2)\cos(2\theta)\right]\left[a^2+b^2+(b^2-a^2)\cos(2\theta)\right] 
$$

Già qua ci basta per calcolare l'ipotetico angolo, infatti invertendo per $\theta$

$$
\begin{align}
\theta&=\frac{1}{2}\left[ 2\pi c_1 - \cos^{-1}\left( -\frac{\sqrt{4a^4+8a^2b^2-A^2+4b^4}}{2\sqrt{a^4-2a^2b^2+b^4}} \right) \right] \\
&=\frac{1}{2}\left[ 2\pi c_1 - \cos^{-1}\left( -\frac{\sqrt{(2a^2+2b^2)^2-A^2}}{2\sqrt{(a^2-b^2)^2}} \right) \right]  \\
&=\frac{1}{2}\left[ 2\pi c_1 - \cos^{-1}\left( -\frac{\sqrt{(a^2+b^2)^2-\frac{A^2}{4}}}{a^2-b^2} \right) \right]  \\
&\hat=\frac{1}{2}\left[  \cos^{-1}\left( \frac{\sqrt{(a^2+b^2)^2-\frac{A^2}{4}}}{a^2-b^2} \right) \right] 
\end{align}
$$

Dove nell'ultimo passagio abbiamo tolto la generalità. Per via delle simmetrie non riusciremo a ricondurci al vero angolo ma sempre ad un angolo compreso tra $\left[0,\frac{\pi}{4}\right]$.

Testiamola per inferire l'angolo, ricordando che 
- $a$ = Asse Maggiore / 2 
- $b$ = Asse Minore / 2

Per fare questo passaggio dobbiamo ricordarci che i nostri chicci non sono ellissi perfetti. Quindi non soltanto questa operazione di inversione sarà un'approssimazione ma potrebbe pure risultare analiticamente impossibile da invertire in alcuni punti. Quindi nella funzione verifichiamo che l'area della bounding box sia compresa tra $\left[ 4ab, 2(a^2+b^2)\right]$ che sono gli estremi per un ellisse. 

Questa funzione spiegata più nel profondo spiega l'istogramma di prima ed anche come l'extent minimo dipende da $a$ e $b$. 

In [ ]:
def recupera_angolo(bounding_box,major_axis,minor_axis):
    a = major_axis / 2.0
    b = minor_axis / 2.0
    if bounding_box >= 4 * a * b and bounding_box <= 2*(a**2+b**2):
        return 0.5 * (np.arccos(((a**2+b**2)**2 - (bounding_box**2*0.25))**0.5 / (a**2-b**2)))
    elif bounding_box > 2*(a**2+b**2): # Area supera estremo superiore
        return 3
    elif bounding_box < 4 * a * b: # Area minore dell'estremo inferiore
        return -3
    
df['BoundingBox'] = df['Area'] / df['Extent']
df['Angle'] = df.apply(lambda row: recupera_angolo(row['BoundingBox'],row["Minor_Axis_Length"],row["Major_Axis_Length"]),axis = 1)



df['Reversed'] = ['Reversed' if x < 3 and x >-3 else "Not Reversed" for x in df['Angle']]

n_reversed=df.value_counts("Reversed") # 70% delle osservazioni da cui abbiamo ottenutto l'angolo
print(n_reversed,sum(n_reversed),sep = '\n')
plt.pie([n_reversed[0],n_reversed[1]], labels =["Reversed","Not Reversed"], autopct='%1.1f%%')
plt.show()


dfTemp = df[df['Reversed'] == "Reversed"]
del dfTemp['Reversed']
pairs_plot(dfTemp,"Class")

distribution(dfTemp,"Angle",'Class')

Ok trenta per cento dei dati persi non è un gran traguardo ma almeno l'angolo risultante sembra abbastanza casuale uniforme. Il fattp che l'angolo sia totalmente incorrleato con il resto è un'ottima cosa per le mie supposizioni

Per farci un'idea di quanto è valida l'approssimazione ad all'ellisse ristimiamo le variabili sotto le nostre ipotesi e vediamo come sono relazionate con le originali

In [ ]:
# Estimation variabile
dfTemp['Expected_Area'] = dfTemp['Major_Axis_Length']  * dfTemp['Minor_Axis_Length'] * np.pi /4
dfTemp['Expected_Eccentricity'] = (1 - (dfTemp['Minor_Axis_Length']/dfTemp['Major_Axis_Length'])**2) ** 0.5 #( a questo punto suppongo che l'abbaino calcolata, non misurata)
dfTemp['Expected_Perimeter'] = ((dfTemp['Major_Axis_Length']**2  + dfTemp['Minor_Axis_Length']**2)/8)**0.5 * np.pi * 2

bivariate(dfTemp,"Area",         "Expected_Area",         'Class')
bivariate(dfTemp,"Eccentricity", "Expected_Eccentricity", 'Class')
bivariate(dfTemp,"Perimeter",    "Expected_Perimeter",    'Class')

Direi buona approssimazione, e nel mentre abbiamo anche risposto ad un altro quesito. Quanto pare _Eccentricity_ è stata proprio ottenuta tale e quale con la formuala che ho implementato io. Non è stata misurata

In [ ]:
pairs_plot(dfTemp,'Class')

# Modellistica
Ok abbiamo messo un sacco di carne sul fuoco, cerchiamo di investire bene. Siamo statistici, non matematici (uffa), quindi applichiamo alla modellistica tutto quello che abbiamo inferito.
Pianificchiamo diverse tecniche di preprocessing e vediamo come si comportano.

1. Dataset base 
2. Dataset base scartando extent
    - Vediamo se scartando l'extent diminuisce l'overfitting dato che è una variabile di merda, non ricordo il nome tecnico
3. Solo due PC del dataset senza extent
4. Aggiungiamo variabili renderle il più possibili commisurabili (ad esempio sotituire extent con bounding box area)
5. prendere solo due o tre PC a seguito del punto 4

Su questi diversi preprocessing stimeremo i seguenti modelli
- LogisticRegression
- RandomForestClassifier
- GradientBoostingClassifier
- LinearDiscriminantAnalysis

Ricarichiamo il dataset per partire da capo

In [ ]:
df = pd.read_csv(link)

In [ ]:
def ASE(target_v,estimated_p):
    ase = 0
    for p,y in zip(estimated_p,target_v):
        ase += (p-y) **2
    return ase / len(estimated_p)

def pre_process_0(df, test):
    return df, test

def pre_process_1(df, test):
    return df.drop("Extent", axis=1), test.drop("Extent", axis=1)

def pre_process_2(df, test):
    df = df.drop("Extent", axis=1)

    scaler = StandardScaler()
    df = scaler.fit_transform(df)

    pca = PCA(n_components = 2)
    pca.fit(df)
    temp = pd.DataFrame(pca.transform(df))
    
    test = test.drop("Extent", axis=1)
    return temp, pd.DataFrame(pca.transform(scaler.transform(test)))

def pre_process_3(df, test):
    df = copy.copy(df)
    df['BoxArea'] = df['Area'] / df['Extent'] #Intenzionato ad essere la versione lineare di Extent
    df['Expected_Area'] = df['Major_Axis_Length']  * df['Minor_Axis_Length'] * np.pi /4
    df['Expected_Perimeter'] = ((df['Major_Axis_Length']**2  + df['Minor_Axis_Length']**2)/8)**0.5 * np.pi * 2
    df['linear_eccentricity'] = np.log(df['Eccentricity']**-1 - 1) #Intenzionato ad essere la versione lineare di Eccentricity
    
    
    test = copy.copy(test)
    test['BoxArea'] = test['Area'] / test['Extent']
    test['Expected_Area'] = test['Major_Axis_Length']  * test['Minor_Axis_Length'] * np.pi /4
    test['Expected_Perimeter'] = ((test['Major_Axis_Length']**2  + test['Minor_Axis_Length']**2)/8)**0.5 * np.pi * 2
    test['linear_eccentricity'] = np.log(test['Eccentricity']**-1 - 1)
    return df, test

def pre_process_4(df, test):
    df,test = pre_process_3(df,test)
    
    scaler = StandardScaler()
    df = scaler.fit_transform(df)

    pca = PCA(n_components = 3)
    pca.fit(df)
    temp = pd.DataFrame(pca.transform(df))
    
    return temp, pd.DataFrame(pca.transform(scaler.transform(test)))

def pre_process_5(df, test):
    df['linear_eccentricity'] = 1000 ** df['Eccentricity'] #Intenzionato ad essere la versione lineare di Eccentricity
    df = df.drop("Eccentricity",axis = 1)
    
    test['linear_eccentricity'] = 1000 ** test['Eccentricity']
    test = test.drop("Eccentricity",axis = 1)

    
    scaler = StandardScaler()
    df = scaler.fit_transform(df)

    pca = PCA(n_components = 3)
    pca.fit(df)
    temp = pd.DataFrame(pca.transform(df))
    
    return temp, pd.DataFrame(pca.transform(scaler.transform(test)))

pre_processes = (pre_process_0,
                 pre_process_1,
                 pre_process_2,
                 pre_process_3,
                 pre_process_4)

models  = (LogisticRegression, 
           RandomForestClassifier, 
           GradientBoostingClassifier, 
           LinearDiscriminantAnalysis)

t_sizes = np.linspace(0.1,1,9,endpoint=False)


In [ ]:
if True:
    Dati = pd.DataFrame({
        "Preprocessing":[],
        "Modello":[],
        "Frazione_Campionamento":[],
        "AUC":[],
        "ASE":[] })

In [ ]:
Feature = df.drop("Class", axis=1)
Target  = df['Class']

reproduce_time = 20

for iter in range(reproduce_time): # facciamo più iter e raccogliamo in seguito i dati medi per mitigare la randomizzazione 
    print(iter)
    for t_size in t_sizes: # iniziamo un ciclo per ogni frazione
        print(f'\t{t_size}')
        X_train_g, X_test_g, y_train, y_test = train_test_split(Feature, Target, test_size=t_size) # Facciamo campionamento GLOBALE per poter confrontare i risultati
        y_test_t = [0 if x == 'Osmancik' else 1 for x in y_test ] #Convertiamo manualmente il target in probabilità
        for pre in pre_processes: # per ogni diversa tecnica di preprocessing
            #print(f'\t\t{pre.__name__}')
            X_train, X_test = pre(X_train_g, X_test_g) # Applichiamo il preprocessing 

            for m in models:
                #print(f'\t\t\t{m.__name__}')
                model = m()
                model.fit(X_train,y_train)
                p_pred = model.predict_proba(X_test)
                p_pred = [p[0] for p in p_pred]
                auc = roc_auc_score(y_test_t, p_pred)
                ase = ASE(y_test_t, p_pred)
                Dati.loc[len(Dati)] = (pre.__name__,m.__name__,t_size,auc, ase)

In [ ]:
Dati_medi = Dati.groupby(by = ["Preprocessing", "Modello", "Frazione_Campionamento"]).agg({'Preprocessing': 'first', 'Modello': 'first', 
                                                                         'Frazione_Campionamento': 'first', 'AUC': 'mean', 'ASE':'mean'})

In [ ]:
def multi_plot(df,filters,level,x,y):
    """
    Return a plot from df dataframe with different level and filters
    filters : dictionary columns, values ex: filters = {'Model':'Regression'}
    levels  : columns for level
    x : x column
    y : y columns
    """
    plt.figure(figsize=(15,5))
    title =' '.join(filters.values())
    
    
    temp = df
    for d in filters:
        temp = temp[temp[d] == filters[d]]

    for l in df[level].unique():
        X = temp[temp[level] == l][x]
        Y = temp[temp[level] == l][y]
        plt.plot(X,Y, label = l)
        
    plt.title(title)
    plt.xticks()
    plt.xlabel(x)
    plt.ylabel(y)
    plt.legend()
    plt.show()

for m in models:
    #multi_plot(Dati_medi,{'Modello':m.__name__},'Preprocessing','Frazione_Campionamento','ASE')
    multi_plot(Dati_medi,{'Modello':m.__name__},'Preprocessing','Frazione_Campionamento','AUC')
    
for p in pre_processes:
    multi_plot(Dati_medi,{'Preprocessing':p.__name__},'Modello','Frazione_Campionamento','ASE')
    #multi_plot(Dati_medi,{'Preprocessing':p.__name__},'Modello','Frazione_Campionamento','AUC')

Non che la differenza sai poi davvero sostanziale, ma _pre\_process\_1_ batte sostanzialmente _pre\_process\_0_. Cioè scartare l'extent migliora il modello

In [ ]:
print(Dati_medi.sort_values(by='AUC',key = lambda x:-x).iloc[0])
print()
print(Dati_medi.sort_values(by='ASE').iloc[0])

# In modelli coì semplici non c'è davvero il rischio di overfitting!
# il rischio arriva con modelli che hanno decision boundary complesse, qua eccetto RF hanno tutti decison boundary lineari, perchè tutta questa complicazione?

Bellissima conclusione, anche inaspettata. I modelli migliori sono la regressione logistica quelli con pre processing 0 o preprocesing 1. Cioè i più semplici. Inoltre considerando che le metriche sono praticamente le stesse scartare l'extent è in definitiva saggio.

In [ ]:
temp = copy.copy(Dati_medi)
temp['Indice'] = list(range(len(temp.Modello)))
temp = temp.set_index('Indice')
del temp['Frazione_Campionamento']
temp.groupby(by=['Modello',"Preprocessing"]).mean().sort_values('ASE')

In [ ]:
temp.groupby(by=['Modello']).mean().sort_values('ASE')

In [ ]:
temp.groupby(by=['Preprocessing']).mean().sort_values('ASE')